<a href="https://colab.research.google.com/github/Alee24/DataScience/blob/main/Advanced_Neural_Networks_Group_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import pandas as pd
from pathlib import Path
from google.colab import files

uploaded = files.upload()

file_name = list(uploaded.keys())[0]
file_path = Path(file_name)

print("File exists:", file_path.exists(), " Path:", file_path)

df = pd.read_csv(file_path)
print("Shape:", df.shape)
display(df.head(10))
print("\nColumns and dtypes:\n", df.dtypes)
print("\nMissing values per column:\n", df.isna().sum())

Saving apartments.csv to apartments (1).csv
File exists: True  Path: apartments (1).csv
Shape: (2520, 7)


,Unnamed: 0,title,location,bedrooms,bathrooms,price,rate
0,0,3 Bedroom Apartment / Flat to rent in Riverside,"Riverside Dr Nairobi, Riverside, Nairobi",3,3,200 000,Per Month
1,1,3 Bedroom Apartment / Flat to rent in Kileleshwa,"Kileleshwa, Nairobi",3,4,70 000,Per Month
2,2,3 Bedroom Apartment / Flat to rent in Nyali,"Links Rd Mombasa, Nyali, Mombasa",3,2,38 000,Per Month
3,3,3 Bedroom Apartment / Flat to rent in Lavington,"Near Valley Arcade, Lavington, Nairobi",3,3,80 000,Per Month
4,4,1 Bedroom Apartment / Flat to rent in Kileleshwa,"Kileleshwa, Nairobi",1,1,110 000,Per Month
5,5,4 Bedroom Apartment / Flat to rent in Kahawa W...,"Thika Rd Nairobi, Kahawa Wendani, Nairobi",4,2,57 750,Per Month
6,6,2 Bedroom Apartment / Flat to rent in Kilimani,"Kilimani, Nairobi",2,3,65 000,Per Month
7,7,3 Bedroom Apartment / Flat to rent in Westlands,NaN,3,3,78 000,Per Month
8,8,1 Bedroom Apartment / Flat to rent in Nyali,"Nyali, Mombasa",1,1,5 000,Per Day
9,9,4 Bedroom Apartment / Flat to rent in Muthaiga,"Muthaiga, Nairobi",4,5,300 000,Per Month



Columns and dtypes:
 Unnamed: 0     int64
title         object
location      object
bedrooms       int64
bathrooms      int64
price         object
rate          object
dtype: object

Missing values per column:
 Unnamed: 0      0
title           0
location      376
bedrooms        0
bathrooms       0
price           0
rate            0
dtype: int64


In [26]:

print("Sample price values (first 50 unique):")
print(df['price'].astype(str).unique()[:50])

print("\nSample rate values (first 50 unique):")
print(df['rate'].astype(str).unique()[:50])

print("\nSample location values (first 50 unique):")
print(df['location'].astype(str).unique()[:50])


Sample price values (first 50 unique):
[' 200 000' ' 70 000' ' 38 000' ' 80 000' ' 110 000' ' 57 750' ' 65 000'
 ' 78 000' ' 5 000' ' 300 000' ' 165 000' ' 55 000' ' 75 000' ' 45 000'
 ' 100 000' ' 125 000']

Sample rate values (first 50 unique):
['Per Month' 'Per Day']

Sample location values (first 50 unique):
['Riverside Dr Nairobi, Riverside, Nairobi' 'Kileleshwa, Nairobi'
 'Links Rd Mombasa, Nyali, Mombasa'
 'Near Valley Arcade, Lavington, Nairobi'
 'Thika Rd Nairobi, Kahawa Wendani, Nairobi' 'Kilimani, Nairobi' 'nan'
 'Nyali, Mombasa' 'Muthaiga, Nairobi' 'Westlands, Nairobi'
 'Kikuyu Town Bus park Kikuyu, Kikuyu, Kikuyu' 'Shanzu, Mombasa'
 'Westlands downtown, Westlands, Nairobi'
 'Kileleshwa Nairobi, Kileleshwa, Nairobi'
 'Grevillea Grove Spring Valley, Spring Valley, Nairobi'
 'Vihiga road, Kileleshwa, Nairobi' 'Off Othaya road, Lavington, Nairobi'
 'Jabavu court, Kilimani, Nairobi']


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor # Import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

data = df.copy()

if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

def clean_price(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().replace(' ', '').replace(',', '').replace('Ksh','').replace('KES','')
    try:
        return int(float(s))
    except:
        return np.nan

data['price_clean'] = data['price'].apply(clean_price)

data['rate'] = data['rate'].fillna('Per Month').str.strip()
data['monthly_price'] = data.apply(lambda row: row['price_clean'] * 30 if row['rate'].lower().startswith('per day') else row['price_clean'], axis=1)

def extract_location_parts(loc):
    if pd.isna(loc) or str(loc).strip().lower() in ['nan','none','']:
        return ("Unknown", "Unknown")
    parts = [p.strip() for p in str(loc).split(',') if p.strip()]
    if len(parts) == 1:
        return (parts[0], parts[0])
    elif len(parts) == 2:
        return (parts[-1], parts[-2])
    else:
        return (parts[-1], parts[-2])

loc_parts = data['location'].apply(extract_location_parts)
data['city'] = loc_parts.apply(lambda x: x[0])
data['area'] = loc_parts.apply(lambda x: x[1])
data['city'] = data['city'].fillna('Unknown')
data['area'] = data['area'].fillna('Unknown')

print("Missing locations after fill:", data['location'].isna().sum())

features = ['bedrooms','bathrooms','city','area']
target = 'monthly_price'

model_df = data.dropna(subset=[target] + ['bedrooms','bathrooms']).copy()
print("Model dataset shape:", model_df.shape)

X = model_df[features]
y = model_df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipelines
numeric_features = ['bedrooms','bathrooms']
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_features = ['city','area']
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Define models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42, n_estimators=100),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'HistGradientBoosting': HistGradientBoostingRegressor(random_state=42) # Add HistGradientBoostingRegressor
}

results = {}
for name, model in models.items():
    pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[name] = {'rmse': rmse, 'mae': mae, 'r2': r2}
    print(f"\n{name} performance on test set:\n RMSE: {rmse:.2f}\n MAE: {mae:.2f}\n R2: {r2:.4f}")

cv = KFold(n_splits=5, shuffle=True, random_state=42)
rf_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestRegressor(random_state=42))])
cv_scores_rf = cross_val_score(rf_pipe, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
print("\nRandomForest 5-fold CV RMSE scores (negative):", cv_scores_rf)
print("Mean RF CV RMSE:", -cv_scores_rf.mean())

print("\nPerforming GridSearchCV for RandomForest...")
param_grid_rf = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10]
}

grid_search_rf = GridSearchCV(rf_pipe, param_grid_rf, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_rf_model = grid_search_rf.best_estimator_
best_params_rf = grid_search_rf.best_params_
best_cv_score_rf = -grid_search_rf.best_score_

print(f"\nBest parameters for RandomForest: {best_params_rf}")
print(f"Best cross-validated RMSE for RandomForest: {best_cv_score_rf:.2f}")

tuned_preds_rf = best_rf_model.predict(X_test)
tuned_rmse_rf = np.sqrt(mean_squared_error(y_test, tuned_preds_rf))
tuned_mae_rf = mean_absolute_error(y_test, tuned_preds_rf)
tuned_r2_rf = r2_score(y_test, tuned_preds_rf)

results['Tuned_RandomForest'] = {'rmse': tuned_rmse_rf, 'mae': tuned_mae_rf, 'r2': tuned_r2_rf, 'best_params': best_params_rf}

print(f"\nTuned RandomForest performance on test set:\n RMSE: {tuned_rmse_rf:.2f}\n MAE: {tuned_mae_rf:.2f}\n R2: {tuned_r2_rf:.4f}")

gb_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', GradientBoostingRegressor(random_state=42))])
cv_scores_gb = cross_val_score(gb_pipe, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
print("\nGradientBoosting 5-fold CV RMSE scores (negative):", cv_scores_gb)
print("Mean GB CV RMSE:", -cv_scores_gb.mean())

print("\nPerforming GridSearchCV for GradientBoosting...")
param_grid_gb = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 4, 5]
}

grid_search_gb = GridSearchCV(gb_pipe, param_grid_gb, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search_gb.fit(X_train, y_train)

best_gb_model = grid_search_gb.best_estimator_
best_params_gb = grid_search_gb.best_params_
best_cv_score_gb = -grid_search_gb.best_score_

print(f"\nBest parameters for GradientBoosting: {best_params_gb}")
print(f"Best cross-validated RMSE for GradientBoosting: {best_cv_score_gb:.2f}")

# Evaluate best GradientBoosting model on test set
tuned_preds_gb = best_gb_model.predict(X_test)
tuned_rmse_gb = np.sqrt(mean_squared_error(y_test, tuned_preds_gb))
tuned_mae_gb = mean_absolute_error(y_test, tuned_preds_gb)
tuned_r2_gb = r2_score(y_test, tuned_preds_gb)

results['Tuned_GradientBoosting'] = {'rmse': tuned_rmse_gb, 'mae': tuned_mae_gb, 'r2': tuned_r2_gb, 'best_params': best_params_gb}

print(f"\nTuned GradientBoosting performance on test set:\n RMSE: {tuned_rmse_gb:.2f}\n MAE: {tuned_mae_gb:.2f}\n R2: {tuned_r2_gb:.4f}")


hgb_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', HistGradientBoostingRegressor(random_state=42))])
cv_scores_hgb = cross_val_score(hgb_pipe, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
print("\nHistGradientBoosting 5-fold CV RMSE scores (negative):", cv_scores_hgb)
print("Mean HGB CV RMSE:", -cv_scores_hgb.mean())

print("\nPerforming GridSearchCV for HistGradientBoosting...")
param_grid_hgb = {
    'model__max_iter': [50, 100, 200], # n_estimators equivalent
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 5, 7]
}

grid_search_hgb = GridSearchCV(hgb_pipe, param_grid_hgb, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search_hgb.fit(X_train, y_train)

best_hgb_model = grid_search_hgb.best_estimator_
best_params_hgb = grid_search_hgb.best_params_
best_cv_score_hgb = -grid_search_hgb.best_score_

print(f"\nBest parameters for HistGradientBoosting: {best_params_hgb}")
print(f"Best cross-validated RMSE for HistGradientBoosting: {best_cv_score_hgb:.2f}")

# Evaluate best HistGradientBoosting model on test set
tuned_preds_hgb = best_hgb_model.predict(X_test)
tuned_rmse_hgb = np.sqrt(mean_squared_error(y_test, tuned_preds_hgb))
tuned_mae_hgb = mean_absolute_error(y_test, tuned_preds_hgb)
tuned_r2_hgb = r2_score(y_test, tuned_preds_hgb)

results['Tuned_HistGradientBoosting'] = {'rmse': tuned_rmse_hgb, 'mae': tuned_mae_hgb, 'r2': tuned_r2_hgb, 'best_params': best_params_hgb}

print(f"\nTuned HistGradientBoosting performance on test set:\n RMSE: {tuned_rmse_hgb:.2f}\n MAE: {tuned_mae_hgb:.2f}\n R2: {tuned_r2_hgb:.4f}")

Missing locations after fill: 376
Model dataset shape: (2520, 10)

LinearRegression performance on test set:
 RMSE: 25510.51
 MAE: 17740.59
 R2: 0.8300

RandomForest performance on test set:
 RMSE: 6026.75
 MAE: 2145.95
 R2: 0.9905

GradientBoosting performance on test set:
 RMSE: 7453.61
 MAE: 4478.95
 R2: 0.9855

HistGradientBoosting performance on test set:
 RMSE: 6039.07
 MAE: 2149.84
 R2: 0.9905

RandomForest 5-fold CV RMSE scores (negative): [-6047.94536471 -5323.32624089 -4636.39622118 -5200.40145408
 -4420.97909098]
Mean RF CV RMSE: 5125.8096743693295

Performing GridSearchCV for RandomForest...

Best parameters for RandomForest: {'model__max_depth': None, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Best cross-validated RMSE for RandomForest: 4988.09

Tuned RandomForest performance on test set:
 RMSE: 6045.20
 MAE: 2148.25
 R2: 0.9905

GradientBoosting 5-fold CV RMSE scores (negative): [-7453.60747523 -6935.19681701 -6174.63179991 -6695.94917375
 -6724.16943643]


In [28]:
print("Hyperparameter tuning results:")

print("\nRandom Forest:")
print(f"Best parameters: {results['Tuned_RandomForest']['best_params']}")
print(f"Tuned performance on test set:\n RMSE: {results['Tuned_RandomForest']['rmse']:.2f}\n MAE: {results['Tuned_RandomForest']['mae']:.2f}\n R2: {results['Tuned_RandomForest']['r2']:.4f}")

print("\nGradient Boosting:")
print(f"Best parameters: {results['Tuned_GradientBoosting']['best_params']}")
print(f"Tuned performance on test set:\n RMSE: {results['Tuned_GradientBoosting']['rmse']:.2f}\n MAE: {results['Tuned_GradientBoosting']['mae']:.2f}\n R2: {results['Tuned_GradientBoosting']['r2']:.4f}")

Hyperparameter tuning results:

Random Forest:
Best parameters: {'model__max_depth': None, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Tuned performance on test set:
 RMSE: 6045.20
 MAE: 2148.25
 R2: 0.9905

Gradient Boosting:
Best parameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 200}
Tuned performance on test set:
 RMSE: 6039.19
 MAE: 2151.82
 R2: 0.9905


In [29]:
# Comparing the results of the different models
import pandas as pd

comparison_data = {
    'Model': [],
    'RMSE': [],
    'MAE': [],
    'R2': []
}

for name, metrics in results.items():
    comparison_data['Model'].append(name)
    comparison_data['RMSE'].append(metrics['rmse'])
    comparison_data['MAE'].append(metrics['mae'])
    comparison_data['R2'].append(metrics['r2'])

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values(by='RMSE') # Sort by RMSE for easier comparison

print("\nModel Performance Comparison:")
display(comparison_df)


Model Performance Comparison:


,Model,RMSE,MAE,R2
1,RandomForest,6026.748937,2145.954080,0.990511
3,HistGradientBoosting,6039.073123,2149.838919,0.990472
5,Tuned_GradientBoosting,6039.189527,2151.820152,0.990471
6,Tuned_HistGradientBoosting,6039.435860,2152.077115,0.990471
4,Tuned_RandomForest,6045.201577,2148.253018,0.990452
2,GradientBoosting,7453.607475,4478.948647,0.985486
0,LinearRegression,25510.505769,17740.590770,0.829977


In [30]:
import joblib

model_filename = 'best_randomforest_apartment_price_model.joblib'
joblib.dump(best_rf_model, model_filename)

print(f"Best RandomForest model exported to {model_filename}")

Best RandomForest model exported to best_randomforest_apartment_price_model.joblib


In [31]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

model_filename = 'best_randomforest_apartment_price_model.joblib'
source_path = model_filename

destination_path = '/content/drive/My Drive/' + model_filename

try:
    shutil.copyfile(source_path, destination_path)
    print(f"Model successfully exported to {destination_path}")
except FileNotFoundError:
    print(f"Error: The model file '{model_filename}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model successfully exported to /content/drive/My Drive/best_randomforest_apartment_price_model.joblib
